In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# import
data = pd.read_csv(
    'input.csv', dtype={
        'item1': str,
        'value1': float,
        'item2': str,
        'value2': float
    })
print('data size: {}'.format(len(data)))

data['shift'] = data['value2'] - data['value1']

x = data[['item1', 'value1', 'item2', 'value2']]
y = data['shift']

data size: 270351


In [3]:
# split training and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=1)

In [4]:
# count feature values
list_items = x_train['item1'].append(x_train['item2']).drop_duplicates().tolist()
n_items = len(list_items)
print("number of unique items: {}".format(n_items))

# columns
item1_col = tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list(
        'item1', vocabulary_list=list_items))
value1_col = tf.feature_column.numeric_column('value1')
item2_col = tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list(
        'item2', vocabulary_list=list_items))
value2_col = tf.feature_column.numeric_column('value2')

number of unique items: 704


In [5]:
# build input function
input_fn_train=tf.estimator.inputs.pandas_input_fn(
        x=x_train,
        y=y_train,
        batch_size=100,
        num_epochs=5,
        shuffle=True
    )

In [6]:
# build model function
def model_fn(features, labels, mode, params):
    input_layer = {
        'item1': tf.feature_column.input_layer(features={'item1': features['item1']},
                                               feature_columns=params['feature_columns'][0]),
        'value1': tf.feature_column.input_layer(features={'value1': features['value1']},
                                                feature_columns=params['feature_columns'][1]),
        'item2': tf.feature_column.input_layer(features={'item2': features['item2']},
                                               feature_columns=params['feature_columns'][2]),
        'value2': tf.feature_column.input_layer(features={'value2': features['value2']},
                                                feature_columns=params['feature_columns'][3])
    }
        
    input_1 = tf.concat((input_layer['item1'],
                         tf.matmul(tf.reshape(input_layer['value1'], shape=[1, n_items]),
                                   input_layer['item1'])),
                        axis=1)
    input_2 = tf.concat((input_layer['item2'],
                         tf.matmul(tf.reshape(input_layer['value2'], shape=[1, n_items]),
                                   input_layer['item2'])),
                        axis=1)
    
    #subtracted = tf.subtract(input_1, input_2)
    subtracted = tf.subtract(input_layer['item1'], input_layer['item2'])
    out = tf.layers.dense(
        inputs=subtracted,
        units=1,
        use_bias=False,
        kernel_initializer=None)

    # define head
    my_head = tf.contrib.estimator.regression_head(
        label_dimension=1,
        loss_fn=None  # custom loss, default: mean_squared_error
    )

    return my_head.create_estimator_spec(
        features=features,
        mode=mode,
        labels=labels,
        optimizer=tf.train.FtrlOptimizer(params['step_size']),
        logits=out,
    )

In [7]:
# custom estimator
regressor = tf.estimator.Estimator(
    model_fn=model_fn,
    params={'feature_columns': [item1_col, value1_col, item2_col, value2_col],
            'step_size': 0.1}
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp8bsboq9s', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f25b2a05f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# train estimator
regressor.train(
    input_fn=input_fn_train,
    steps=None
)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp8bsboq9s/model.ckpt.
INFO:tensorflow:loss = 0.74005055, step = 1
INFO:tensorflow:global_step/sec: 315.701
INFO:tensorflow:loss = 0.36239582, step = 101 (0.318 sec)
INFO:tensorflow:global_step/sec: 334.197
INFO:tensorflow:loss = 0.8801305, step = 201 (0.299 sec)
INFO:tensorflow:global_step/sec: 340.169
INFO:tensorflow:loss = 0.77202576, step = 301 (0.294 sec)
INFO:tensorflow:global_step/sec: 336.837
INFO:tensorflow:loss = 0.3163119, step = 401 (0.297 sec)
I

INFO:tensorflow:loss = 0.31268027, step = 6901 (0.305 sec)
INFO:tensorflow:global_step/sec: 329.632
INFO:tensorflow:loss = 0.29462737, step = 7001 (0.303 sec)
INFO:tensorflow:global_step/sec: 316.641
INFO:tensorflow:loss = 0.3759687, step = 7101 (0.316 sec)
INFO:tensorflow:global_step/sec: 338.255
INFO:tensorflow:loss = 0.3426169, step = 7201 (0.296 sec)
INFO:tensorflow:global_step/sec: 345.836
INFO:tensorflow:loss = 0.5700729, step = 7301 (0.290 sec)
INFO:tensorflow:global_step/sec: 340.085
INFO:tensorflow:loss = 0.2657693, step = 7401 (0.294 sec)
INFO:tensorflow:global_step/sec: 296.227
INFO:tensorflow:loss = 0.16125631, step = 7501 (0.339 sec)
INFO:tensorflow:global_step/sec: 310.517
INFO:tensorflow:loss = 0.2794549, step = 7601 (0.320 sec)
INFO:tensorflow:global_step/sec: 335.551
INFO:tensorflow:loss = 0.5232483, step = 7701 (0.298 sec)
INFO:tensorflow:global_step/sec: 320.628
INFO:tensorflow:loss = 0.15727223, step = 7801 (0.312 sec)
INFO:tensorflow:global_step/sec: 337.322
INFO:t

In [9]:
# evaluation

# evaluation input function
input_fn_eval=tf.estimator.inputs.pandas_input_fn(
        x=x_test,
        y=y_test,
        batch_size=1,
        num_epochs=1,
        shuffle=False
    )

# evaluate
eval_result = regressor.evaluate(input_fn=input_fn_eval)
print(eval_result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-05-16:31:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp8bsboq9s/model.ckpt-10814
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-05-16:32:18
INFO:tensorflow:Saving dict for global step 10814: average_loss = 0.32169634, global_step = 10814, label/mean = 0.022441076, loss = 0.32169634, prediction/mean = 0.007838105
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10814: /tmp/tmp8bsboq9s/model.ckpt-10814
{'average_loss': 0.32169634, 'label/mean': 0.022441076, 'loss': 0.32169634, 'prediction/mean': 0.007838105, 'global_step': 10814}


In [10]:
print(eval_result)

{'average_loss': 0.32169634, 'label/mean': 0.022441076, 'loss': 0.32169634, 'prediction/mean': 0.007838105, 'global_step': 10814}


In [11]:
# construct prediction table

# list existing items with value 0
x_pred = pd.DataFrame({'item1': list_items})
x_pred['value1'] = 0
x_pred['item2'] = ''
x_pred['value2'] = 0

# list existing items with value 1
x_pred_ext = x_pred.copy()
x_pred_ext['value1'] = 1
x_pred = x_pred.append(x_pred_ext, ignore_index=True)

# swap 1 <--> 2
x_pred2 = x_pred[['item2', 'value2', 'item1', 'value1']]
x_pred2.columns = ['item1', 'value1', 'item2', 'value2']
x_pred = x_pred.append(x_pred2, ignore_index=True).drop_duplicates()

In [12]:
# prediction
predictions = list(
    regressor.predict(
        input_fn=tf.estimator.inputs.pandas_input_fn(
            x=x_pred,
            batch_size=len(x_pred),
            num_epochs=1,
            shuffle=False)))

# output
pred = []
for i in predictions:
    pred.append(np.float64(i['predictions'][0]))
x_pred['shift'] = pred
x_pred.to_csv('predictions.csv', sep=',', index=False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp8bsboq9s/model.ckpt-10814
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
